Учим conv сеть для классификации

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 9.8 MB/s 


In [3]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=356eef86b509556a1a7125c07b548ce339064c684893b08c46ae65d23d9ed514
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [4]:
import pandas as pd
import re
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping 

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
data = pd.read_excel("/content/drive/MyDrive/nero_final/less7 Сверточные нейронные сети для анализа текста/отзывы за лето.xlsx")
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [7]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [8]:
data['target'] = data['target'].astype(int)
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично всё,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [9]:
data['target'].value_counts()

1    16724
0     3024
Name: target, dtype: int64

In [10]:
data_positiv_f = data['target'].isin([1])
data_positiv = data[data_positiv_f]

In [11]:
data_negativ_f = data['target'].isin([0])
data_negativ = data[data_negativ_f]

In [12]:
data_positiv.shape

(16724, 5)

In [13]:
data_negativ.shape

(3024, 5)

In [14]:
data_corpus_positiv = " ".join(data_positiv["text"])
data_corpus_positiv = data_corpus_positiv.lower()

data_corpus_negativ = " ".join(data_negativ["text"])
data_corpus_negativ = data_corpus_negativ.lower()

In [15]:
tokens_positiv = word_tokenize(data_corpus_positiv)
tokens_negativ = word_tokenize(data_corpus_negativ)

In [16]:
tokens_not_filtered = ['всё','все','в','я','на','и','с','что','приложение','по','это','у','а']

tokens_filtered_positiv = [word for word in tokens_positiv if word.isalnum() and word not in tokens_not_filtered]
tokens_filtered_negativ = [word for word in tokens_negativ if word.isalnum() and word not in tokens_not_filtered]

In [17]:
max_words = 400

dist = FreqDist(tokens_filtered_positiv)
tokens_filtered_top_positiv = [pair[0] for pair in dist.most_common(max_words-1)]

In [18]:
tokens_filtered_top_positiv[:10]

['очень',
 'удобно',
 'удобный',
 'отлично',
 'работать',
 'спасибо',
 'нравиться',
 'хороший',
 'отличный',
 'хорошо']

In [19]:
dist = FreqDist(tokens_filtered_negativ)
tokens_filtered_top_negativ = [pair[0] for pair in dist.most_common(max_words-1)]

In [20]:
tokens_filtered_top_negativ[:10]

['после',
 'работать',
 'телефон',
 'мочь',
 'обновление',
 'антивирус',
 'раз',
 'быть',
 'не',
 'прошивка']

In [21]:
vocabulary_positiv = {v: k for k, v in dict(enumerate(tokens_filtered_top_positiv, 1)).items()}
vocabulary_negativ = {v: k for k, v in dict(enumerate(tokens_filtered_top_negativ, 1)).items()}

In [22]:
max_len = 40

def text_to_sequence(text, maxlen, positiv=1):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if positiv == 1 and word in vocabulary_positiv:
          result.append(vocabulary_positiv[word])
        elif word in vocabulary_negativ:
          result.append(vocabulary_negativ[word])  
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [23]:
x_data = np.asarray([text_to_sequence(row['text'], max_len,row['target']) for index, row in data.iterrows()], dtype=np.int32)

In [24]:
x_data.shape

(19748, 40)

In [25]:
num_classes = 2
epochs = 20
batch_size = 512
print_batch_n = 100

In [26]:
x_train, x_val, y_train, y_val = train_test_split(x_data, data['target'], test_size=0.33, random_state=42)

In [27]:
import tensorflow as tf

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)

In [28]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [29]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
x_train.shape

(13231, 40)

In [31]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
24/24 [==============================] - 5s 161ms/step - loss: 0.5692 - accuracy: 0.8385 - val_loss: 0.4125 - val_accuracy: 0.8557
Epoch 2/20
24/24 [==============================] - 4s 150ms/step - loss: 0.4112 - accuracy: 0.8487 - val_loss: 0.3777 - val_accuracy: 0.8588


In [32]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 40ms/step - loss: 0.3983 - accuracy: 0.8446


Test score: 0.3982779085636139
Test accuracy: 0.8445603847503662


 Рассмотреть 2-а варианта сеточек
 
 1 Инициализировать tf.keras.layers.Embedding предобученными векторами

In [93]:
data_path = tf.keras.utils.get_file(
    "ruscorpora_mystem",
    "https://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz",
    untar=True,
)

In [94]:
import os
import pathlib
import tarfile

In [85]:
data_dir = pathlib.Path(data_path).parent # / 'ruscorpora_mystem'
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir)#/ "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of directories: 2
Directory names: ['keras.json', 'datasets']
Number of files in comp.graphics: 2
Some example filenames: ['keras.json', 'datasets']


**БИЛСЯ ДВА ДНЯ НЕ СМОГ ПОНЯТЬ ПОЧЕМУ НЕ ПРИНИМАЕТ colab БИБЛИОТЕКУ. ГОНЯЛ ЧЕРЕЗ tirfile. ПРИЧЕМ В КОРНЕ datasets ВИДЕТ СКАЧАННЫЕ ФАЙЛЫ, НО ОБРАТИТЬСЯ К НИМ НЕ МОЖЕТ. Я ЗДАЮСЬ )))))**

In [96]:
os.listdir(data_dir/'datasets')

['ruscorpora_mystem_cbow_300_2_2015.bin.gz.tar.gz',
 'ruscorpora_mystem_cbow_300_2_2015.tar.gz',
 'ruscorpora_mystem.tar.gz']

In [97]:
os.listdir(data_dir/'datasets'/'ruscorpora_mystem.tar.gz') #а так нет!

NotADirectoryError: ignored

In [100]:
tar = tarfile.open(data_path, "r:gz") #и так нет!

FileNotFoundError: ignored

In [101]:
tar = tarfile.open('/root/.keras/datasets/ruscorpora_mystem.tar.gz', "r:gz") #и так нет!

ReadError: ignored

In [98]:
print(open(data_dir + "comp.graphics").read())

TypeError: ignored

In [99]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir #/ dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath #/ fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing datasets, 2 files found


IsADirectoryError: ignored